In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# The 8-Queens Problem

The <a href="https://en.wikipedia.org/wiki/Eight_queens_puzzle">eight queens puzzle</a> is the problem of placing eight chess queens on a chessboard so that no two queens can capture each other.  In <a href="https://en.wikipedia.org/wiki/Chess">chess</a> a queen can capture another piece if this piece is either
<ol>
    <li>in the same row,</li>
    <li>in the same column, or</li>
    <li>in the same diagonal.</li>
</ol>
The image below shows a queen in row 3, column 4.  All the locations where a piece can be captured by this queen are marked with an arrow.

<img src="queen-captures.png">

We will solve this puzzle by encoding it as a *constrant satisfaction problem*.
- The set of values is $\{1,\cdots,8\}$.  These values are interpreted as columns.
- We will use 8 variables $Q_1$, $\cdots$, $Q_8$.  If $Q_i = j$, then the interpretation 
  is that there is a queen at the position $(i, j)$, i.e. the queen in row $i$ is 
  placed in column $j$. 
- The condition 
  $$ \bigl\{ \texttt{Q}_i \not= \texttt{Q}_j \;\bigm|\; 
      i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\} 
  $$
  specifies that there is at most on queen in a given column.
- The condition
  $$ \bigl\{ |\texttt{Q}_i - \texttt{Q}_j| \not= |i - j| \;\bigm|\; 
     i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}
  $$
  specifies that there is at most one queen in a given diagonal.

In [ ]:
import * as BCS from "./02-Backtracking-Constraint-Solver";
type NQueensCSP = BCS.CSP<string, number>;

In [ ]:
function queensCSP(n: number = 8): NQueensCSP {
    const variables: string[] = [];
    for (let i = 1; i <= n; i++) {
        variables.push(`Q${i}`);
    }

    const values: number[] = [];
    for (let i = 1; i <= n; i++) {
        values.push(i);
    }
    
    const constraints: string[] = [];
    for (let i = 1; i <= n; i++) {
        for (let j = i + 1; j <= n; j++) {
            constraints.push(`Q${i} != Q${j}`);
            const diff = j - i;
            constraints.push(`Math.abs(Q${i} - Q${j}) != ${diff}`);
        }
    }
    
    return {
        variables,
        values,
        constraints
    };
}


Next, we create a *CSP* that encodes the *8-queens-puzzle*.

In [ ]:
const CSP = queensCSP();
CSP;

This *CSP* has 8 variables and 56 constraints.  When we solved the same problem using propositional logic and the *Davis-Putnam algorithm*, we had used 64 propositional variables and 512 clauses.

In [ ]:
[CSP.variables.length, CSP.constraints.length]

On my desktop computer (2023 MacStudio with M2 Max processor) it takes about 50 ms to solve the problem. 

In [ ]:
console.time("solution");
const solution = await BCS.solve(CSP);
console.timeEnd("solution");
solution

The function `showSolution(solution, n, width)` takes a record `solution` that contains a variable assignment representing a solution to the n-queens puzzle. It displays this solution on a chess board.

* `solution` is a record (dictionary) mapping the variables $\texttt{Q}_i$ to numbers.
  If $\texttt{solution['Q'}_i\texttt{]} = k$, then the queen in row $i$ is placed in column $k$.
* `n` specifies the size of the board (default is 8).
* `width` specifies the display width of the board (default is "50%").

In [ ]:
import { display } from 'tslab';
function showSolution(solution: BCS.Assignment<string, number>, n: number = 8, width = "50%") {
    const boardArray: string[][] = Array.from({ length: n }, () => Array(n).fill(''));
    for (let col = 1; col <= n; col++) {
        const rowVal = solution[`Q${col}`];
        if (rowVal !== undefined) {
            const row = Number(rowVal);
            if (row - 1 < n && col - 1 < n) {
                boardArray[row - 1][col - 1] = '♕';
            }
        }
    }
    let html = `<div style="display: grid; grid-template-columns: repeat(${n}, 1fr); width: ${width}; aspect-ratio: 1/1; border: 2px solid #8b4513;">`;
    for (let row = 0; row < n; row++) {
        for (let col = 0; col < n; col++) {
            const piece = boardArray[row][col];
            const bgColor = (row + col) % 2 === 0 ? '#ffce9e' : '#d18b47';
            let cellContent = '';
            if (piece) {
                cellContent = `
                <svg viewBox="0 0 100 100" style="width: 80%; height: 80%; display: block;">
                    <text x="50%" y="55%" font-size="90" text-anchor="middle" dominant-baseline="middle" fill="black">
                        ${piece}
                    </text>
                </svg>`;
            }
            html += `<div style="
                display: flex; 
                align-items: center; 
                justify-content: center; 
                background-color: ${bgColor}; 
                overflow: hidden;
            ">${cellContent}</div>`;
        }
    }
    html += `</div>`;
    display.html(html);
}

In [ ]:
if (solution) {
    showSolution(solution, 8);
}

If we would use a brute force approach that checks permutations, we would have to check $8! = 40\,320$ different configurations.  

In [ ]:
function factorial(n: number): number {
    if (n === 0 || n === 1) return 1;
    let result = 1;
    for (let i = 2; i <= n; i++) {
        result *= i;
    }
    return result;
}

In [ ]:
factorial(8);